In [8]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision 
import matplotlib.pyplot as plt 
from skimage.color import rgb2lab, lab2rgb, rgb2gray, xyz2lab
from skimage.io import imsave
import numpy as np
import os

In [ ]:
images = img_to_array(load_img('woman.jpg'))